# Hotel Base Chunk

In [1]:
import pandas as pd
import numpy as np

from collections import defaultdict

# For memory efficiency
dtypes = {
    'site_name': np.uint8,
    'posa_continent': np.uint8,
    'user_location_country': np.uint8,
    'user_location_region': np.uint16,
    'user_location_city': np.uint16,
    'orig_destination_distance': np.float16,
    'user_id': np.uint32,
    'is_mobile': np.uint8,
    'is_package': np.uint8,
    'channel': np.uint8,
    'srch_adults_cnt': np.uint8,
    'srch_children_cnt': np.uint8,
    'srch_rm_cnt': np.uint8,
    'srch_destination_id': np.uint16,
    'srch_destination_type_id': np.uint16,
    'is_booking': np.uint8,
    'cnt': np.uint8,
    'hotel_continent': np.uint8,
    'hotel_country': np.uint8,
    'hotel_cluster': np.uint8
}

df_train = pd.read_csv('../dataset/hotel_train.csv', dtype=dtypes, chunksize=50000, iterator=True)
df_dest = pd.read_csv('../dataset/destinations.csv', dtype=np.float32)

In [2]:
def transform(df):
    df = pd.merge(df, df_dest, how='left', on='srch_destination_id')
    del df['is_booking']
    del df['cnt']
    
    ### Parse Date & Time
    from datetime import datetime

    def parse_date(d):
        if type(d) is int:
            d = datetime.fromtimestamp(d / 1000000000).isoformat()
        if str(d) == 'nan':
            d = '2013-01-01'

        try:
            return np.datetime64(str(d))
        except:
            return np.datetime64('2013-01-01')


    df['date_time'] = df['date_time'].map(parse_date)
    df['srch_co'] = df['srch_co'].map(parse_date)
    df['srch_ci'] = df['srch_ci'].map(parse_date)

    ### Make new features with time data and others
    df['days'] = (df['srch_co'] - df['srch_ci']).map(lambda d: d.days)
    df['month'] = df['date_time'].map(lambda d: d.month - 1)
    df['hour'] = df['date_time'].map(lambda d: d.hour - 1)

    df['people'] = df['srch_adults_cnt'] + df['srch_children_cnt']

    del df['srch_co']
    del df['srch_ci']
    del df['date_time']
    
    ### Fill empty values in column 'orig_destination_distance'
    means = df[['posa_continent', 'hotel_continent', 'orig_destination_distance']].groupby(['posa_continent', 'hotel_continent']).mean()
    means = means['orig_destination_distance'].reset_index().fillna(2000)

    means['avg_distance'] = means['orig_destination_distance']

    df = pd.merge(df, means, how='left')
    df['orig_destination_distance'] = df['orig_destination_distance'].fillna(df['avg_distance'])

    del means
    del df['avg_distance']
    df = df.fillna(-1)

    ### One-Hot Encoding
    from sklearn.preprocessing import OneHotEncoder as ohe

    categorical_columns = {'site_name': 54, 'posa_continent': 5, 'user_location_country': 240, 'user_location_region': 1028,
                    'channel': 11, 'srch_destination_type_id': 10,
                    'hotel_continent': 7, 'hotel_country': 213, 'hotel_market': 2118, 'month': 12, 'hour': 24}

    for c, v in categorical_columns.items():
        columns = [c + str(i) for i in range(v)]
        one_hot = np.zeros([df.shape[0], v], dtype=np.uint8)
        one_hot[np.arange(df.shape[0]), df[c].values] = 1
        np.put(one_hot, [v], 1)
        df = pd.concat([df, pd.DataFrame(one_hot, columns=columns, dtype=np.uint8)], axis=1)

        del df[c]

    del df['srch_destination_id']
    del df['user_location_city']
    del df['user_id']
    
    return df

### Build Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm

clfs = list()

for d in tqdm(df_train):
    d = transform(d)
    
    X = d.drop('hotel_cluster', axis=1)
    y = d['hotel_cluster']
    
    clf = RandomForestClassifier(max_depth=20, n_estimators=2, n_jobs=4)
    clf.fit(X, y)
    
    clfs.append(clf)

475it [36:04,  4.47s/it]

### Validation